In [1]:
import time
import requests
import pymongo

In [2]:
import pandas as pd
import numpy as np

In [3]:
import pickle

In [4]:
# next_sequence_number=0
# api_url='https://hxobin8em5.execute-api.us-west-2.amazonaws.com/api/'
# api_key='vYm9mTUuspeyAWH1v-acfoTlck-tCxwTw9YfCynC'
# stor={}
# payload = {'api_key': api_key,
#                        'sequence_number': next_sequence_number}
# response = requests.post(api_url, json=payload)
# data = response.json()
# stor[next_sequence_number] = data
# next_sequence_number = data['_next_sequence_number']

In [5]:
def get_data():
    api_url='https://hxobin8em5.execute-api.us-west-2.amazonaws.com/api/'
    api_key='vYm9mTUuspeyAWH1v-acfoTlck-tCxwTw9YfCynC'
    next_sequence_number=0
    payload = {'api_key': api_key,
                   'sequence_number': 0}
    response = requests.post(api_url, json=payload)
    data = response.json()
    stor=pd.DataFrame.from_dict(data['data'])
    arr=[0,]
    for i in range(1500,1800):
        
        """Fetch data from the API."""
        payload = {'api_key': api_key,
                   'sequence_number': i}
        response = requests.post(api_url, json=payload)
        temp_data = response.json()
        temp_df=pd.DataFrame.from_dict(temp_data['data'])
        stor=pd.concat([stor,temp_df])
        next_sequence_number = data['_next_sequence_number']
        arr.append(next_sequence_number)
    return stor

In [6]:
a = get_data()

In [38]:
a=a.drop_duplicates('sequence_number')

In [39]:
df=a

In [40]:
df.shape

(12, 44)

In [41]:
if 'acct_type' not in df.columns:
                df['acct_type'] = df.iloc[:, 0].apply(lambda x: 'unknown')
df['is_fraud'] = df['acct_type'].apply(lambda x: 1 if 'fraud' else ('unknown' if 'unknown' else 0))     

In [42]:
required_columns=['body_length', 'channels', 'delivery_method', 'event_published',
       'fb_published', 'has_analytics', 'has_logo',
       'name_length', 'num_order', 'num_payouts', 'org_facebook',
       'org_twitter', 'sale_duration', 'sale_duration2', 'show_map',
       'user_age', 'user_created', 'user_type']

In [43]:
len(required_columns)

18

In [44]:
for col in required_columns:
        if col not in df.columns:
                df[col] = df.iloc[:, 0].apply(lambda x: 0)

In [45]:
X=df[required_columns]

In [46]:
X.shape

(12, 18)

In [47]:
df.acct_type.nunique()

1

In [16]:
model_rf = pickle.load(open('rf.pkl','rb'))

In [48]:
y_rf = model_rf.predict(X)

In [49]:
y_rf

array([0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1])

In [50]:
sum(y_rf)

4

In [51]:
y_rf_prob = model_rf.predict_proba(X)

In [52]:
len(y_rf_prob)

12

In [53]:
test=df.reset_index()

In [54]:
test.to_json('test_script_examples.json')